In [1]:
import pandas as pd
import os
import itertools
import xlsxwriter
from rapidfuzz import fuzz

os.chdir('M:\GY number duplicates')
pd.set_option('display.max_columns', 1000, 'display.width', 1000, 'display.max_rows',1000)
demogs = df = pd.read_csv('GY_update.txt', '\t')

C:\Users\CHowitt01\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [56]:
demogs = demogs.dropna(subset = ['SocialSecurityNumber']) # Ensure no NaNs in SSN column
demogs = demogs.drop_duplicates() # Ensure no duplicates


demogs = demogs[demogs['SocialSecurityNumber'].str.contains('GY')]
demogs.fillna('', inplace=True)

demogs_pairs = demogs[demogs.groupby("SocialSecurityNumber")['SocialSecurityNumber'].transform('size') == 2]
demogs_more_than_2_instances = demogs[demogs.groupby("SocialSecurityNumber")['SocialSecurityNumber'].transform('size') > 2]


groups_paris = demogs_pairs.groupby('SocialSecurityNumber') # Groupby SSN


master_list = []
for group_name, df_group in groups_paris:
    gy_group_list = df_group.astype(str).values.tolist() # Each group as a list (group) of lists (rows)
    master_list.append(gy_group_list)

cols = demogs.columns.tolist()
cols.append('Ratio')
cols.append('Diffs')
cols.append('NoInstances')

df_results = pd.DataFrame(columns = cols)
master_score = []

for i in range(len(master_list)): # For each list (group) in master_list
    number_of_rows = len(master_list[i]) # Define number of rows in group

    if number_of_rows == 2:
        score_list = []
        avg_score = []
        diffs_list = []
        for x, y in zip(master_list[i][0][1:], master_list[i][1][1:]): # For each row in group minus URN
            score = fuzz.ratio(x, y)
            score_list.append(score)

            splitB = set(x.split(' '))
            splitA = set(y.split(' '))
            diff = splitB.symmetric_difference(splitA)
            if diff != '':
                diff = ", ".join(diff) 
                diffs_list.append(diff)
            
    avg_score = sum(score_list)/len(score_list)
    
    
    master_list[i][0].append(int(avg_score))
    master_list[i][1].append(int(avg_score))


    diffs_list = list(filter(None, diffs_list))

    diffs_string = "        ".join(diffs_list)
    master_list[i][0].append(diffs_string)
    master_list[i][1].append(diffs_string)
    
    master_list[i][0].append('')
    master_list[i][1].append('')

    df_results.loc[len(df_results)] = master_list[i][0]
    df_results.loc[len(df_results)] = master_list[i][1]
    
    

demogs_more_than_2_instances['NoInstances'] = demogs_more_than_2_instances['SocialSecurityNumber'].map(demogs_more_than_2_instances['SocialSecurityNumber'].value_counts())
df_results = df_results.append(demogs_more_than_2_instances.astype(str))

df_results['NoInstances'] = df_results['SocialSecurityNumber'].map(df_results['SocialSecurityNumber'].value_counts())

df_results.sort_values(by= ['Ratio', 'SocialSecurityNumber'], ascending = [False, False], inplace = True) # sort within the groups (ratio)

df_results = df_results[['URN',
 'FirstName',
 'LastName',
 'DOB',
 'Address1',
 'Address2',
 'Gender',
 'SocialSecurityNumber',
 'Ratio',
 'Diffs',
 'NoInstances']]


writer = pd.ExcelWriter("GY_Number_Dups.xlsx", engine='xlsxwriter') # Create a Pandas Excel writer using XlsxWriter as the engine

# Convert the dataframe to an XlsxWriter Excel object. Turn off the default
# header and index and skip one row to allow us to insert a user defined
# header
df_results.to_excel(writer, sheet_name='Sheet1', startrow=1, header=False, index=False)

workbook = writer.book # Get the xlsxwriter workbook and worksheet objects
worksheet = writer.sheets['Sheet1']

(max_row, max_col) = df_results.shape # Get the dimensions of the dataframe

column_settings = [] # Create a list of column headers, to use in add_table() 
for header in df_results.columns:
    column_settings.append({'header': header})

worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings,
                                                'style': 'Table Style Light 11'}) # Add the table

worksheet.set_column(0, max_col - 1, 12) # Make the columns wider for clarity.

writer.save() # Close the Pandas Excel writer and output the Excel file

C:\Users\CHOWIT~1\AppData\Local\Temp/ipykernel_6796/3492265620.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demogs_more_than_2_instances['NoInstances'] = demogs_more_than_2_instances['SocialSecurityNumber'].map(demogs_more_than_2_instances['SocialSecurityNumber'].value_counts())
